In [1]:
import pandas as pd 
import numpy as np
import os
import requests
import time
from bs4 import BeautifulSoup

## Read Data

In [2]:
os.path.abspath('')

'C:\\Users\\yeqiu\\Desktop\\opgg\\v0.0.0'

In [3]:
search_params = 'all'
fldr = os.path.join(os.path.abspath(''), '..', 'data')

for f in os.listdir(fldr):
    
    if search_params in f:
        
        print(f)
        fp = os.path.join(fldr, f)

fp

api-all_20210925.csv


'C:\\Users\\yeqiu\\Desktop\\opgg\\v0.0.0\\..\\data\\api-all_20210925.csv'

In [4]:
df = pd.read_csv(fp)

C:\Users\yeqiu\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Helpers

In [5]:
#No header/user-agent cycling?
headers = {
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'Referer': '',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36'
}

In [6]:
def accessor_id(user_name: 'str') -> 'int': 
    
    """ Get accesssor summoner ID to feed into OPGG API """
    
    def html_safe(user_name: "str") -> "str": 
        
        """Return HTML safe username to search"""

        html_str = user_name
        html_str.replace(' ', '+')

        return html_str
        
    
    if not isinstance(user_name, str): 
        return;
    
    base_url = 'https://na.op.gg/summoner/'
    url_substring = 'userName={}'.format(html_safe(user_name)) 
    url = base_url + url_substring
    
    try:
        r = requests.get(url, headers=headers)
        #print(f'Fetching data for {user_name} @ {url} ...')
    except ConnectionError:
        print(f'Connection error for {user_name} @ {url}')

    if r.ok:
    
        soup = BeautifulSoup(r.text, 'html.parser')
        
        #OPGG accesssor ID (for anything involving API requests)
        divs = soup.findAll("div", {"data-summoner-id" : True})
        ids = set(
                    [x['data-summoner-id'] for x in divs]
                 )
       
        
        #Check uniqueness
        if len(ids) != 1:
            
            #print(f'Multiple or no IDs found, ids: {ids} for user {user_name}\nMost likely due to name change')
            
            return 
        
        
        accesssor_id = ids.pop()        
        return accesssor_id
    
    else:
        
        print(f'Error for user {user_name}, url: {url}')
        return 

## Scrape OPgg

In [7]:
assert df.shape[0] == len(set(df.summonerName.values)), 'Oh no there are duplicate names...'

In [8]:
def print_progress(iteration, total, prefix ='', suffix ='', decimals =1, length =100, fill ='█', printEnd="\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end =printEnd)

    if iteration == total:
        print()

In [9]:
#Runtime: 5 hrs.
start: 10:35 am
note: do not turn off computer

SyntaxError: invalid syntax (Temp/ipykernel_8472/2454680801.py, line 2)

In [ ]:
main = []

print_progress(0, len(df.summonerName.values), prefix ='', suffix ='Complete')

for i,name in enumerate(df.summonerName.values):
    
    user_id = accessor_id(name)
    
    if user_id:
    
        tmp = pd.DataFrame({'user_name': name, 'user_id': user_id}, index=[0])

        main.append(tmp)
    
    #  if i%100==0: 
    #      print("Scraped {} of {}, {} remaining".format(i,len(names),len(names)-i))
    
    if i%100 ==0: 
        time.sleep(10) #Politeness is key. 100 requests/2 min. is pretty polite.
                       #Considering most scrapers run at 32/sec ...
    
    print_progress(i, len(df.summonerName.values), prefix ='', suffix ='Complete')

In [ ]:
frame = pd.concat(main)

In [ ]:
frame.shape

In [ ]:
frame.reset_index(drop=True)

In [ ]:
# setting up filepath 
data_folder = 'data/'
base_filename = 's2-user_ids'
timestamp = str(date.today()).replace('-','')
ext = '.csv'
filepath = '_'.join([data_folder+base_filename,timestamp+ext])

#export
frame.to_csv(filepath,index=False,header=True)